# Manipulating DataFrames with Pandas - 2 (Grouping Data)

We can use `groupby()` function to
* Split data into groups
* Apply a function to each group
  - Aggregate (e.g. compute counts, means)
  - Transform (e.g. fill NAs within groups with group specific values)
  - Filter (filter out data based on a group statistic)
* Combine the results

>Aggregating functions are the ones that reduce the dimension of the returned objects. Some common aggregating functions are tabulated below:
- mean()	Compute mean of groups
- sum()	Compute sum of group values
- size()	Compute group sizes
- count()	Compute count of group
- std()	Standard deviation of groups
- var()	Compute variance of groups
- sem()	Standard error of the mean of groups
- describe()	Generates descriptive statistics
- first()	Compute first of group values
- last()	Compute last of group values
- nth()	Take nth value, or a subset if n is a list
- min()	Compute min of group values
- max()	Compute max of group values.
[source](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)